In [2]:
import requests

url = "https://kata.geosci.ai/challenge/birthquakes"  
r = requests.get(url)
r.status_code

200

In [3]:
from IPython.display import Markdown

Markdown(r.text)

# Birthquakes

We are going to look at earthquakes, on your birthdate. Birthquakes!

We will also be implementing the haversine formula for determining the distance between two ponts on the earth's surface.

This challenge is a bit different from the previous ones. You can use any old string for your key, as usual, but if you use a date, you'll get data for that date. For example:

      url = 'https://kata.geosci.ai/challenge/birthquakes'
      params = {'key': '1980-06-30'}  # <-- The key can be a date.
      r = requests.get(url, params)

Your challenge input is now `r.text`. There is a header row containing the names of the columns, plus a number of data rows or 'records'. Each row has 13 columns, and represents the data for a single earthquake.

You need to answer the following questions:

1. How many records (i.e. earthquakes) are there?
2. What is the depth **in metres** of the earthquake with the largest **Magnitude**? (If there's more than one, give the deepest.)
3. What is the great circle distance **to the nearest km**, as given by the haversine formula, between the epicentres of the two **largest** earthquakes, as measured by magnitude?
4. Consider all pairs of events. How many pairs are less than 100 km of each other? (Exactly 100 km would **not** be included.)

Note that because we're asking about epicentres, so you don't need to worry about depth when calculating great circle distances.

For Question 4, only count unique pairs. For example, inthe diagram below there are 15 pairs of points altogether, of which there are 7 pairs with a mutual distance of < 100 km here &mdash; 1 pair on the left and 6 on the right:

      
      x                  x
                            x
         x              x  x
            ==========
              100 km


## Haversine formula

There are several formulas for computing [great circle distance](https://en.wikipedia.org/wiki/Great-circle_distance) on a sphere. The simplest accurate one is the haversine formula, which is described here.

Given two points with (_latitude_, _longitude_), we'll denote point 1 with $(\varphi_1, \lambda_1)$ and point 2 with $(\varphi_2, \lambda_2)$. Then distance _d_ is related to radius _r_ by:

$$   d  = 2r \arcsin\left(\sqrt{\sin^2\left(\frac{\varphi_2 - \varphi_1}{2}\right) + \cos(\varphi_1) \cos(\varphi_2)\sin^2\left(\frac{\lambda_2 - \lambda_1}{2}\right)}\right)$$

Some hints about implementing this in Python:

- Use $r = 6371\ \mathrm{km}$ for the radius of the earth.
- $\sin^2(x)$ means $\sin(x) \times \sin(x)$.
- Both the `math` module and NumPy have the functions `sin()`, `cos()`; these functions expect radians, so an angle in degrees must be converted to radians with `radians()` before giving it to the function.
- The arcsine function in `math` is called `asin()`; in NumPy it's `arcsin()`.
- The function should return distances **to the nearest km**.
- You should get the following results from your function:
  - The distance from (0, 0) to (0, 1) is 111 km.
  - The distance from (0, 2.35) to (90, 2.35) is 10008 km. [(Why?)](https://en.wikipedia.org/wiki/History_of_the_metre)
  - The distance from (44.65, -63.58) to (53.73, -1.86) is 4448 km.


## A quick reminder how this works

You can retrieve your data by choosing any date (or any old Python string to choose a random date) as a **`<KEY>`** and substituting here:
    
    https://kata.geosci.ai/challenge/birthquakes?key=<KEY>
                                                     ^^^^^
                                                     you can use a date, e.g. 2001-01-01

To answer question 1, make a request like:

    https://kata.geosci.ai/challenge/birthquakes?key=<KEY>&question=1&answer=123
                                                     ^^^^^          ^        ^^^
                                                     your key       Q        your answer

[Complete instructions at kata.geosci.ai](https://kata.geosci.ai/challenge)

----

© 2020 Agile Scientific, licensed CC-BY

In [4]:
my_key = "1995-10-21"
params = {'key': my_key}
r = requests.get(url, params)
birthquakes = r.text

In [5]:
from io import StringIO
import pandas as pd 
import numpy as np

birthquakes_test = birthquakes.replace("|",",")
data = birthquakes_test.split("\n")
data2 = []

for i in data:
    data2.append(i.split(","))

In [6]:
df = pd.DataFrame(data2) 
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df

,#EventID,Time,Latitude,Longitude,Depth/km,Author,Catalog,Contributor,ContributorID,MagType,Magnitude,MagAuthor,EventLocationName,NaN
1,ci3239418,1995-10-21T23:59:41.810,35.748,-117.608,9.034,ci,ci,ci,ci3239418,mc,0.93,ci,15km NNE of Ridgecrest,California
2,ak995dilpghk,1995-10-21T23:02:12.400,65.1732,-148.4747,6.3,ak,ak,ak,ak995dilpghk,ml,1.8,ak,39 km S of Livengood,Alaska
3,ak995dilp12d,1995-10-21T23:00:12.530,65.0308,-147.2743,-0.9,ak,ak,ak,ak995dilp12d,ml,0.8,ak,18 km ENE of Fox,Alaska
4,ak995dila3dy,1995-10-21T22:30:44.220,62.6897,-151.267,90.1,ak,ak,ak,ak995dila3dy,ml,2,ak,33 km NW of Petersville,Alaska
5,ci3239412,1995-10-21T22:25:07.700,34.984,-116.953,5.021,ci,ci,ci,ci3239412,mc,1.61,ci,11km NE of Barstow,California
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,ci3239258,1995-10-21T00:27:31.150,35.036,-117.679,-0.935,ci,ci,ci,ci3239258,mc,1.94,ci,5km NNW of Boron,California
213,ci3239257,1995-10-21T00:20:36.700,35.772,-117.633,4.672,ci,ci,ci,ci3239257,mc,1.13,ci,17km NNE of Ridgecrest,California
214,usp00075ga,1995-10-21T00:02:32.190,-26.745,178.028,656.8,us,us,us,usp00075ga,mb,4.5,us,south of the Fiji Islands,None
215,ci3239255,1995-10-21T00:00:13.030,35.742,-117.605,4.14,ci,ci,ci,ci3239255,mc,1.15,ci,15km NNE of Ridgecrest,California


In [7]:
# 1) How many records (i.e. earthquakes) are there?

params = {'key': "1995-10-21",   
              'question': 1,   
              'answer': 215,  
             }
r = requests.get(url, params)
r.text

'Correct'

2) What is the depth in metres of the earthquake with the largest Magnitude? (If there's more than one, give the deepest.)

In [8]:
# Change some columns from object to float.
df[["Latitude", "Longitude","Depth/km","Magnitude"]] = df[["Latitude", "Longitude","Depth/km","Magnitude"]].apply(pd.to_numeric)

In [9]:
df.loc[df['Magnitude'].idxmax()]

0
#EventID                                       usp00075gf
Time                              1995-10-21T02:38:57.120
Latitude                                            16.84
Longitude                                         -93.469
Depth/km                                            159.3
Author                                                 us
Catalog                                                us
Contributor                                            us
ContributorID                                  usp00075gf
MagType                                                mw
Magnitude                                             7.2
MagAuthor                                             hrv
EventLocationName    13 km NW of Ocozocoautla de Espinosa
NaN                                                Mexico
Name: 193, dtype: object

In [10]:
params = {'key': "1995-10-21",   
              'question': 2,   
              'answer': 159300,  
             }
r = requests.get(url, params)
r.text

'Correct'

3) What is the great circle distance to the nearest km, as given by the haversine formula, between the epicentres of the two largest earthquakes, as measured by magnitude?

In [11]:
import numpy as np
import math

# phi1 = 16.84
# lambda1 = -93.469
# phi2 = -5.037
# lambda2 = 153.318

phi1 = 44.65
lambda1 = -63.58
phi2 = 53.73
lambda2 = -1.86

phi1,phi2,lambda1,lambda2,dphi,dlambda = map(math.radians,[phi1,phi2,lambda1,lambda2,(phi2-phi1),(lambda2-lambda1)])
r = 6371
x = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
d = 2 * r* np.arcsin(math.sqrt(x))
round(d)

4448

In [12]:
def great_circle(phi1,phi2,lambda1, lambda2):
    
    phi1,phi2,lambda1,lambda2,dphi,dlambda = map(math.radians,[phi1,phi2,lambda1,lambda2,(phi2-phi1),(lambda2-lambda1)])
    r = 6371
    x = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    d = 2 * r* np.arcsin(math.sqrt(x))
    
    return(round(d))

In [13]:
df.sort_values(by='Magnitude',ascending=False)


,#EventID,Time,Latitude,Longitude,Depth/km,Author,Catalog,Contributor,ContributorID,MagType,Magnitude,MagAuthor,EventLocationName,NaN
193,usp00075gf,1995-10-21T02:38:57.120,16.840,-93.469,159.3,us,us,us,usp00075gf,mw,7.2,hrv,13 km NW of Ocozocoautla de Espinosa,Mexico
27,usp00075hd,1995-10-21T18:52:47.680,-5.037,153.318,33.0,us,us,us,usp00075hd,mw,5.5,hrv,139 km ESE of Kokopo,Papua New Guinea
210,usp00075gb,1995-10-21T00:34:06.750,-55.118,146.410,10.0,us,us,us,usp00075gb,mw,5.2,hrv,west of Macquarie Island,None
54,usp00075h3,1995-10-21T15:36:18.920,-2.893,-77.935,33.0,us,us,us,usp00075h3,mb,5.0,us,54 km SSE of Sucúa,Ecuador
23,usp00075hh,1995-10-21T19:03:44.740,37.505,72.820,33.0,us,us,us,usp00075hh,mb,4.9,us,75 km NE of Khand?d,Afghanistan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,usp00075gv,1995-10-21T12:16:11.090,-31.432,-69.160,110.0,us,us,us,usp00075gv,,NaN,us,27 km ESE of Calingasta,Argentina
133,usp00075gq,1995-10-21T09:18:56.480,-31.238,-68.485,90.5,us,us,us,usp00075gq,,NaN,us,21 km N of San Martín,Argentina
148,usp00075gk,1995-10-21T07:46:26.090,-31.172,-68.242,80.0,us,us,us,usp00075gk,,NaN,us,37 km NE of San Martín,Argentina
168,usp00075gh,1995-10-21T06:02:23.270,-31.071,-68.358,100.0,us,us,us,usp00075gh,,NaN,us,42 km NNE of San Martín,Argentina


In [14]:
params = {'key': "1995-10-21",   
              'question': 3,   
              'answer': 12638,  
             }
r = requests.get(url, params)
r.text

'Correct'

4) Consider all pairs of events. How many pairs are less than 100 km of each other? (Exactly 100 km would not be included.)

In [23]:
pairs = 0

for i in range(len(lat)-1):
    
    for j in range(i+1,len(lat)):
        
        gc = great_circle(lat[i],lat[j],long[i], long[j]) #Usar esta funcion para saber si es mas
        
        if gc < 100:
           
          pairs = pairs + 1

In [24]:
pairs

3869

In [26]:
params = {'key': "1995-10-21",   
              'question': 4,   
              'answer': 3869,  
             }
r = requests.get(url, params)
r.text

'Correct! The next challenge is: https://kata.geosci.ai/challenge/fossil-hunting - good luck!'